# This script retrieve the wanted datasets from online
### Output : csv files for each parameter, combining all Tau

- data here : https://turbulence.oden.utexas.edu/
- adapt the following list with the wanted data

#### Channel Flow

In [1]:
parameters = ["vel_fluc","mean","vp"]
Re_tau = ['5200', '2000', '1000', '0550', '0180']

In [2]:
def parse_file(obj):
    data = {}
    lines = obj.split("\n")
    for line in lines:
        if line.startswith('%'):
            if 'nx = ' in line:
                data['nx'] = (line.split('=')[1].strip())
            elif 'ny = ' in line:
                data['ny'] = (line.split('=')[1].strip())
            elif 'nz = ' in line:
                data['nz'] = (line.split('=')[1].strip())
            elif 'Lx = ' in line:
                try:
                    data['Lx'] = (line.split('=')[1].strip())
                except ValueError:
                    print(f"Error parsing Lx: {line.split('=')[1].strip()}")
                    data['Lx'] = None
            elif 'Lz = ' in line:
                try:
                    data['Lz'] = (line.split('=')[1].strip())
                except ValueError:
                    print(f"Error parsing Lz: {line.split('=')[1].strip()}")
                    data['Lz'] = None
            elif 'n = ' in line and "U_mean" not in line:
                data['n'] = line.split('=')[1].strip()
            elif 'nu = ' in line:
                data['nu'] = line.split('=')[1].strip()
            elif 'delta = ' in line and not "y/" in line:
                data['delta'] = line.split('=')[1].strip() 
            elif 'U_mean = ' in line:
                data['U_mean'] = line.split('=')[1].strip()
            elif 'u_tau = ' in line:
                data['u_tau'] = line.split('=')[1].strip()
            elif 'Re_tau = ' in line and "," not in line:
                data['Re_tau'] = line.split('=')[1].strip()

    return data

In [3]:
import requests
import io
import pandas as pd
import re


def skip_header(content):
    """Find dynamiccly the number of lines to skip in the header of the file."""
    lines = content.split("\n")
    lines_to_skip = 0
    for line in lines:
        if line.strip().startswith("%"):
            lines_to_skip += 1
        else:
            break
    return lines_to_skip


def get_header(content):
    """Get dynamiccly the header of the dataset."""
    lines = content.split("\n")
    line = lines[skip_header(content) - 2]
    header = re.split(r"\s+", line.strip())[1:]
    return header


def download_and_combine_data(parameter, Re_tau):
    frames = []  # List to store data frames for each Re_tau

    for tau in Re_tau:
        url = f"https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_{tau}_{parameter}_prof.dat"
        display(url)
        response = requests.get(url)

        if response.status_code == 200:
            file_content = response.text
            file_like_object = io.StringIO(file_content)

            data = parse_file(file_content)

            df = pd.read_csv(
                file_like_object,
                sep=r"\s+",
                skiprows=skip_header(file_content),
                names=get_header(file_content),
            )

            for key, value in data.items():
                df[key] = value
            
            frames.append(df)
        else:
            print(
                f"Failed to download the file for Re_tau={tau}. Status code: {response.status_code}"
            )

    # Concatenate all data frames
    combined_df = pd.concat(frames, ignore_index=True)

    # Save the combined data frame to a CSV file
    combined_df.to_csv(f"{parameter}_combined_channel.csv", index=False)
    print(
        f"Combined data for parameter '{parameter}' and Re_tau={Re_tau} saved to '{parameter}_combined_channel.csv'"
    )


# Loop through each parameter and process
for id, param in enumerate(parameters):
    id = download_and_combine_data(param, Re_tau)
    display(id)

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_5200_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_2000_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_1000_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_0550_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_0180_vel_fluc_prof.dat'

...

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_1000_vp_prof.dat'

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_0550_vp_prof.dat'

'https://turbulence.oden.utexas.edu/channel2015/data/LM_Channel_0180_vp_prof.dat'

Combined data for parameter 'vp' and Re_tau=['5200', '2000', '1000', '0550', '0180'] saved to 'vp_combined_channel.csv'


None

#### Couette Flow

In [4]:
parameters = ["vel_fluc","mean","vp"]
Re_tau = ['0093', '0220', '0500']
Lx = ['100PI', '020PI']

In [5]:
def download_and_combine_data_couette():
    for parameter in parameters:
        frames = []  # List to store data frames for each Re_tau
        for lx in Lx:
            for tau in Re_tau:
                url = f"https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R{tau}_{lx}_{parameter}_prof.dat"
                display(url)
                response = requests.get(url)

                if response.status_code == 200:
                    file_content = response.text
                    file_like_object = io.StringIO(file_content)

                    data = parse_file(file_content)

                    df = pd.read_csv(
                        file_like_object,
                        sep=r"\s+",
                        skiprows=skip_header(file_content),
                        names=get_header(file_content),
                    )

                    for key, value in data.items():
                        df[key] = value
                    
                    frames.append(df)
                else:
                    print(
                        f"Failed to download the file for Re_tau={tau}. Status code: {response.status_code}"
                    )

        # Concatenate all data frames
        combined_df = pd.concat(frames, ignore_index=True)

        # Save the combined data frame to a CSV file
        combined_df.to_csv(f"{parameter}_combined_couette.csv", index=False)

        display(f"Saved combined data for parameter '{parameter}' to '{parameter}_combined_couette.csv'.")

In [6]:
download_and_combine_data_couette()

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0093_100PI_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0220_100PI_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0500_100PI_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0093_020PI_vel_fluc_prof.dat'

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0220_020PI_vel_fluc_prof.dat'

...

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0500_100PI_vp_prof.dat'

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0093_020PI_vp_prof.dat'

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0220_020PI_vp_prof.dat'

'https://turbulence.oden.utexas.edu/couette2018/data/LM_Couette_R0500_020PI_vp_prof.dat'

"Saved combined data for parameter 'vp' to 'vp_combined_couette.csv'."